In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
train = pd.read_csv('../data/train.csv')
train.head()

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [4]:
book_example = pd.read_parquet('../data/book_train.parquet/stock_id=0')
trade_example =  pd.read_parquet('../data/trade_train.parquet/stock_id=0')
stock_id = '0'
book_example = book_example[book_example['time_id']==5]
book_example.loc[:,'stock_id'] = stock_id
trade_example = trade_example[trade_example['time_id']==5]
trade_example.loc[:,'stock_id'] = stock_id

In [5]:
book_example.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0


In [8]:
trade_example.head()

,time_id,seconds_in_bucket,price,size,order_count,stock_id
0,5,21,1.002301,326,12,0
1,5,46,1.002778,128,4,0
2,5,50,1.002818,55,1,0
3,5,57,1.003155,121,5,0
4,5,68,1.003646,4,1,0


In [10]:
book_example['wap'] = (book_example['bid_price1'] * book_example['ask_size1'] +
                                book_example['ask_price1'] * book_example['bid_size1']) / (
                                       book_example['bid_size1']+ book_example['ask_size1'])

In [11]:
book_example.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id,wap
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0,1.001434
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0,1.001448
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0,1.001448
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0,1.001443
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0,1.001443


In [23]:
fig = px.line(book_example, x="seconds_in_bucket", y="wap", title='WAP of stock_id_0, time_id_5')
fig.show()

In [13]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

In [14]:
book_example.loc[:,'log_return'] = log_return(book_example['wap'])
book_example = book_example[~book_example['log_return'].isnull()]

In [22]:
fig = px.line(book_example, x="seconds_in_bucket", y="log_return", title='Log return of stock_id_0, time_id_5')
fig.show()

In [16]:
def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))
realized_vol = realized_volatility(book_example['log_return'])
print(f'Realized volatility for stock_id 0 on time_id 5 is {realized_vol}')

Realized volatility for stock_id 0 on time_id 5 is 0.004499364172786558


In [19]:
import os
from sklearn.metrics import r2_score
import glob
list_order_book_file_train = glob.glob('../data/book_train.parquet/*')

In [20]:
list_order_book_file_train

['../data/book_train.parquet/stock_id=17',
 '../data/book_train.parquet/stock_id=28',
 '../data/book_train.parquet/stock_id=10',
 '../data/book_train.parquet/stock_id=26',
 '../data/book_train.parquet/stock_id=19',
 '../data/book_train.parquet/stock_id=21',
 '../data/book_train.parquet/stock_id=75',
 '../data/book_train.parquet/stock_id=81',
 '../data/book_train.parquet/stock_id=86',
 '../data/book_train.parquet/stock_id=72',
 '../data/book_train.parquet/stock_id=44',
 '../data/book_train.parquet/stock_id=88',
 '../data/book_train.parquet/stock_id=43',
 '../data/book_train.parquet/stock_id=20',
 '../data/book_train.parquet/stock_id=27',
 '../data/book_train.parquet/stock_id=18',
 '../data/book_train.parquet/stock_id=11',
 '../data/book_train.parquet/stock_id=16',
 '../data/book_train.parquet/stock_id=29',
 '../data/book_train.parquet/stock_id=89',
 '../data/book_train.parquet/stock_id=42',
 '../data/book_train.parquet/stock_id=73',
 '../data/book_train.parquet/stock_id=87',
 '../data/b

In [21]:
def realized_volatility_per_time_id(file_path, prediction_column_name):
    df_book_data = pd.read_parquet(file_path)
    df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
                                      df_book_data['bid_size1']+ df_book_data[
                                  'ask_size1'])
    df_book_data['log_return'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return)
    df_book_data = df_book_data[~df_book_data['log_return'].isnull()]
    df_realized_vol_per_stock =  pd.DataFrame(df_book_data.groupby(['time_id'])['log_return'].agg(realized_volatility)).reset_index()
    df_realized_vol_per_stock = df_realized_vol_per_stock.rename(columns = {'log_return':prediction_column_name})
    stock_id = file_path.split('=')[1]
    df_realized_vol_per_stock['row_id'] = df_realized_vol_per_stock['time_id'].apply(lambda x:f'{stock_id}-{x}')
    return df_realized_vol_per_stock[['row_id',prediction_column_name]]


In [24]:
def past_realized_volatility_per_stock(list_file,prediction_column_name):
    df_past_realized = pd.DataFrame()
    for file in list_file:
        df_past_realized = pd.concat([df_past_realized,
                                     realized_volatility_per_time_id(file,prediction_column_name)])
    return df_past_realized
df_past_realized_train = past_realized_volatility_per_stock(list_file=list_order_book_file_train,
                                                           prediction_column_name='pred')

In [25]:
df_past_realized_train.head()

,row_id,pred
0,17-5,0.004091
1,17-11,0.002155
2,17-16,0.002566
3,17-31,0.002221
4,17-62,0.002155


In [26]:
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train.head()

,stock_id,time_id,target,row_id
0,0,5,0.004136,0-5
1,0,11,0.001445,0-11
2,0,16,0.002168,0-16
3,0,31,0.002195,0-31
4,0,62,0.001747,0-62


In [27]:
train = train[['row_id','target']]
df_joined = train.merge(df_past_realized_train[['row_id','pred']], on = ['row_id'], how = 'left')
df_joined

,row_id,target,pred
0,0-5,0.004136,0.004499
1,0-11,0.001445,0.001204
2,0-16,0.002168,0.002369
3,0-31,0.002195,0.002574
4,0-62,0.001747,0.001894
...,...,...,...
428927,126-32751,0.003461,0.003691
428928,126-32753,0.003113,0.004104
428929,126-32758,0.004070,0.003118
428930,126-32763,0.003357,0.003661


In [28]:
from sklearn.metrics import r2_score
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))
R2 = round(r2_score(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
RMSPE = round(rmspe(y_true = df_joined['target'], y_pred = df_joined['pred']),3)
print(f'Performance of the naive prediction: R2 score: {R2}, RMSPE: {RMSPE}')

Performance of the naive prediction: R2 score: 0.628, RMSPE: 0.341


In [32]:
list_order_book_file_test = glob.glob('../data/book_test.parquet/*')
df_naive_pred_test = past_realized_volatility_per_stock(list_file=list_order_book_file_test,
                                                           prediction_column_name='target')
df_naive_pred_test.to_csv('../output/tutorial_submission.csv',index = False)

In [33]:
df_naive_pred_test

,row_id,target
0,0-4,0.000294
